In [ ]:
# %pip install sdv

### 23.09.18 New Idea
- tps, latency를 모두 generate하지 않고 config만 generate한 후에 성능 prediction
- Optimization epoch 1500

### Data Prepare (Metric)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

A_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")

In [3]:
A_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")
# AA_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")
# B_metrics = pd.read_csv("/home/sein/mk_config/ycsb_B/results/external_metrics.csv")

# metrics = pd.concat([A_metrics, AA_metrics, B_metrics], axis=0)
metrics = A_metrics.drop(['Unnamed: 0'], axis = 1)
# metrics.reset_index(inplace=True)
# metrics = metrics.drop(['index'], axis='columns')

In [4]:
metrics = metrics.replace([np.inf],9999999)

metrics


,tps,latency
0,0.216666,20847397
1,0.322221,12744942
2,3789.313102,1076
3,0.166667,12510551
4,0.266666,10686198
...,...,...
995,0.138889,11836756
996,0.149999,16386967
997,0.227778,19293708
998,0.188888,10529195


### Data Prepare (Config)

In [5]:
import random
import glob
import os
import sys
import pandas as pd

knob_list = glob.glob("/home/sein/mk_config/ycsb_AA/configs/my_*.cnf")


In [6]:
cnt = 0

for xx in range(len(knob_list)):
    path = "/home/sein/mk_config/ycsb_AA/configs/my_{}.cnf".format(xx)
    # knob_list = glob.glob("/home/sein/2023_EDBT/KCC_tpcc_dataset/my_*.cnf")
    a_all = pd.read_csv(path, sep="=", names=['Sample', 'value'], header=2)
    a_all = a_all.set_index("Sample")
    cur_all_df = a_all.T
    
    if cnt == 0:
        A_config = cur_all_df
    else :
        A_config = pd.concat([A_config, cur_all_df], axis=0)
    cnt += 1
A_config = A_config.reset_index()
A_config = A_config.drop(["index"],axis=1)
A_config = A_config.drop(A_config.columns[[0,1]], axis=1)


A_config

Sample,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,18000,4276224,945000,610000,1,4096000,432013312,0,29,...,159744,520000,91200,8000,63,7680,163577856,57344,104448,1
996,1,57000,5636096,245000,211000,1,8716288,268435456,1,24,...,455936,940000,89600,2000,29,11520,230686720,69632,61440,0
997,1,2000,622592,45000,537000,1,393216,671088640,7,20,...,308480,810000,66000,8000,48,12544,536870912,81920,108544,1
998,0,38000,5505024,700000,598000,0,9125888,834666496,3,17,...,404992,260000,70400,2000,57,3840,218103808,36864,63488,1


In [7]:
samples = pd.concat([A_config,metrics], axis=1)

samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,0.216666,20847397
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,0.322221,12744942
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3789.313102,1076
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,0.166667,12510551
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,0.266666,10686198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,18000,4276224,945000,610000,1,4096000,432013312,0,29,...,91200,8000,63,7680,163577856,57344,104448,1,0.138889,11836756
996,1,57000,5636096,245000,211000,1,8716288,268435456,1,24,...,89600,2000,29,11520,230686720,69632,61440,0,0.149999,16386967
997,1,2000,622592,45000,537000,1,393216,671088640,7,20,...,66000,8000,48,12544,536870912,81920,108544,1,0.227778,19293708
998,0,38000,5505024,700000,598000,0,9125888,834666496,3,17,...,70400,2000,57,3840,218103808,36864,63488,1,0.188888,10529195


In [8]:
### boolean에 해당하는 열 정리

discrete_columns = [samples.columns[0], samples.columns[5],samples.columns[10],samples.columns[13],
                    samples.columns[17],samples.columns[24],samples.columns[30],samples.columns[31],
                    samples.columns[32],samples.columns[34],samples.columns[36],samples.columns[37],
                    samples.columns[58],samples.columns[60],samples.columns[64],samples.columns[68],
                    samples.columns[72],samples.columns[73],samples.columns[74],samples.columns[75],
                    samples.columns[77],samples.columns[80],samples.columns[82],samples.columns[83],
                    samples.columns[90],samples.columns[91],samples.columns[92],samples.columns[93],
                    samples.columns[118],samples.columns[123],samples.columns[124],samples.columns[125],
                    samples.columns[126]]

In [9]:
### continuous 열 정리
all_columns = samples.columns
continuous_columns = all_columns.drop(discrete_columns)

In [10]:
for i in range(len(discrete_columns)):
    a = discrete_columns[i]
    samples = samples.astype({a:'int'})

In [11]:
for i in range(len(continuous_columns)):
    a = continuous_columns[i]
    samples = samples.astype({a:'float'})

### config - metric prediction (with raw data)

In [13]:

### TabNet
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

X_all = np.array(A_config)
Y_all = np.array(metrics)

X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler = MinMaxScaler().fit(y_train)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)
scaled_y_train = Y_scaler.transform(y_train)
scaled_y_test = Y_scaler.transform(y_test)

# Tabnet 모델 생성
regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) ### Basic

# 모델 학습
regressor.fit(X_train=scaled_X_train, y_train=scaled_y_train,
              eval_set=[(scaled_X_test, scaled_y_test)],
              patience=50, 
              batch_size = 64,
              max_epochs=10000,
              eval_metric=['mse'])

predictions = regressor.predict(scaled_X_test)

test_score = mean_squared_error(y_pred = predictions, y_true = scaled_y_test)
# 성능 평가
print('BEST VALID SCORE : ', regressor.best_cost)
# print('MSE_SCORE : ', test_score)
print('R2 SCORE : ' , r2_score(predictions, scaled_y_test))


epoch 0  | loss: 0.86403 | val_0_mse: 0.16616 |  0:00:00s
epoch 10 | loss: 0.05097 | val_0_mse: 0.05912 |  0:00:01s
epoch 20 | loss: 0.04892 | val_0_mse: 0.06108 |  0:00:03s
epoch 30 | loss: 0.04105 | val_0_mse: 0.05393 |  0:00:05s
epoch 40 | loss: 0.02155 | val_0_mse: 0.02933 |  0:00:07s
epoch 50 | loss: 0.0184  | val_0_mse: 0.01921 |  0:00:08s
epoch 60 | loss: 0.01209 | val_0_mse: 0.05181 |  0:00:10s
epoch 70 | loss: 0.01489 | val_0_mse: 0.02547 |  0:00:12s
epoch 80 | loss: 0.01097 | val_0_mse: 0.01242 |  0:00:13s
epoch 90 | loss: 0.01075 | val_0_mse: 0.01272 |  0:00:15s
epoch 100| loss: 0.01003 | val_0_mse: 0.02712 |  0:00:17s
epoch 110| loss: 0.0074  | val_0_mse: 0.01594 |  0:00:19s
epoch 120| loss: 0.00924 | val_0_mse: 0.01199 |  0:00:20s
epoch 130| loss: 0.00797 | val_0_mse: 0.00758 |  0:00:22s
epoch 140| loss: 0.00759 | val_0_mse: 0.01069 |  0:00:24s
epoch 150| loss: 0.01206 | val_0_mse: 0.01094 |  0:00:25s
epoch 160| loss: 0.00713 | val_0_mse: 0.00912 |  0:00:27s
epoch 170| los

### LHS SAMPLING

In [14]:
import pandas as pd

knob_info = pd.read_csv('Knob_Information_MySQL_v5.7.csv')

knob_min = knob_info['raw_min']
knob_max = knob_info['raw_max']


In [15]:
import numpy as np

def lhs_sampling(min_val, max_val, num_samples):
    # LHS 샘플 생성
    lhs_samples = np.linspace(min_val, max_val, num_samples, endpoint=False)
    np.random.shuffle(lhs_samples)
    return lhs_samples
samples = lhs_sampling(knob_min, knob_max, 1000)

# 생성된 LHS 샘플 출력
print(samples)
# samples.shape

[[1.71000000e-01 1.02608290e+04 1.79646054e+06 ... 2.32622080e+04
  2.32622080e+04 1.71000000e-01]
 [9.42000000e-01 5.65200580e+04 9.87782349e+06 ... 1.23529216e+05
  1.23529216e+05 9.42000000e-01]
 [6.56000000e-01 3.93603440e+04 6.88006758e+06 ... 8.63354880e+04
  8.63354880e+04 6.56000000e-01]
 ...
 [6.95000000e-01 4.17003050e+04 7.28885248e+06 ... 9.14073600e+04
  9.14073600e+04 6.95000000e-01]
 [8.24000000e-01 4.94401760e+04 8.64098714e+06 ... 1.08183552e+05
  1.08183552e+05 8.24000000e-01]
 [6.79000000e-01 4.07403210e+04 7.12114586e+06 ... 8.93265920e+04
  8.93265920e+04 6.79000000e-01]]


In [17]:
samples.shape

(1000, 138)

In [18]:
metrics.shape

(1000, 2)

In [16]:
### LHS sampling으로 생성한 X값에 대한 scaling

from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

new_X = np.array(samples)
Z_scaler = MinMaxScaler().fit(new_X)

predictions_new = regressor.predict(new_X)


In [20]:
predictions_new.shape

(1000, 2)

### 실제로 확인

In [295]:
# knob 실제로 벤치마킹할 때 predict한 metric값이 나오는지 확인

# all_test[500]
# all_test_pd = pd.DataFrame(all_test)
inverse_transformed_ex = Z_scaler.inverse_transform(all_test) 
# print(inverse_transformed_ex)
ex_A = inverse_transformed_ex[15] #500번째 테스트 샘플

inverse_transformed_ex_m = Y_scaler.inverse_transform(predictions) 
ex_A_met = inverse_transformed_ex_m[15] #500번째 성능

# ex_A , ex_A_met
# ex_A



NameError: name 'all_test' is not defined

In [181]:
ex_A_p = pd.DataFrame(ex_A)
ex_A_met_p = pd.DataFrame(ex_A_met)

pd.options.display.float_format = '{:.5f}'.format
# real_ex_A = [math.exp(x) for x in ex_A_p]  # 각 값에 대한 지수 함수 값을 계산하여 실수로 변환

print(ex_A_p)
print(ex_A_met_p)

                  0
0           0.99800
1       16966.00200
2     9156698.11200
3      938120.00000
4      101897.89790
..              ...
133     14051.84000
134 644630972.41600
135     81758.20800
136     94021.63200
137         0.00000

[138 rows x 1 columns]
               0
0        7.94501
1 12350178.00000


In [89]:
ex_A_p[0][0]

0.9318637274549098

In [174]:
for i in range (len(A_config)):
    print('{} = {}'.format(A_config.columns[i],round(ex_A_p[0][i])))

automatic_sp_privileges  = 1
back_log  = 55800
binlog_cache_size  = 9752044
binlog_group_commit_sync_delay  = 930000
binlog_group_commit_sync_no_delay_count  = 930000
binlog_rows_query_log_events  = 1
binlog_stmt_cache_size  = 9752044
bulk_insert_buffer_size  = 998579896
default_week_format  = 7
div_precision_increment  = 28
end_markers_in_json  = 1
eq_range_index_dive_limit  = 9300
expire_logs_days  = 7
explicit_defaults_for_timestamp  = 1
flush_time  = 167
ft_min_word_len  = 15
ft_query_expansion_limit  = 930
general_log  = 1
group_concat_max_len  = 3810
innodb_adaptive_hash_index_parts  = 476
innodb_adaptive_max_sleep_delay  = 930000
innodb_autoextend_increment  = 930
innodb_buffer_pool_size  = 16052440269
innodb_change_buffer_max_size  = 46
innodb_cmp_per_index_enabled  = 1
innodb_commit_concurrency  = 930
innodb_compression_failure_threshold_pct  = 93
innodb_compression_level  = 8
innodb_compression_pad_pct_max  = 70
innodb_concurrency_tickets  = 93000
innodb_deadlock_detect  = 1


IndexError: index 138 is out of bounds for axis 0 with size 138

In [21]:
# a = Z_scaler.data_max_
# b = Z_scaler.data_min_
# (new_X[0]- b) / (a - b)

### metric 확인

In [393]:
# metrics
# metrics.shape 1000,2
# new_pred = predictions[200:701] #scaling 되어있음

# inverse_new_pred = Y_scaler.inverse_transform(new_pred) 
# # print(inverse_new_pred)
# inverse_new_pred_pd = pd.DataFrame(inverse_new_pred)
# inverse_new_pred_pd.rename(columns={0: "tps", 1:"latency"}, inplace=True)
# # inverse_new_pred_pd

# new_metrics_re = pd.concat([metrics,inverse_new_pred_pd], axis=0)

# new_metrics_re = new_metrics_re.reset_index()
# new_metrics_re = new_metrics_re.drop(['index'], axis=1)
# new_metrics_re

,tps,latency
0,0.21667,20847397.00000
1,0.32222,12744942.00000
2,3789.31310,1076.00000
3,0.16667,12510551.00000
4,0.26667,10686198.00000
...,...,...
1495,2723.86743,1233149.25000
1496,6.25019,11350298.00000
1497,-19.29882,13902865.00000
1498,-41.08655,15584320.00000


In [17]:
### new_metrics_re = 원래 metric + predic_metric

predictions_new_df = pd.DataFrame(predictions_new)
inverse_new_pred = Y_scaler.inverse_transform(predictions_new) 
# print(inverse_new_pred)
inverse_new_pred_pd = pd.DataFrame(inverse_new_pred)
inverse_new_pred_pd.rename(columns={0: "tps", 1:"latency"}, inplace=True)
# inverse_new_pred_pd

new_metrics_re = pd.concat([metrics,inverse_new_pred_pd], axis=0)
# new_metrics_re = new_metrics_re.drop(['index'], axis=1)

new_metrics_re = new_metrics_re.reset_index()


In [18]:
new_metrics_re = new_metrics_re.drop(['index'], axis=1)

new_metrics_re


,tps,latency
0,2.166658e-01,2.084740e+07
1,3.222207e-01,1.274494e+07
2,3.789313e+03,1.076000e+03
3,1.666666e-01,1.251055e+07
4,2.666662e-01,1.068620e+07
...,...,...
1995,1.336532e+07,6.192648e+09
1996,1.966208e+07,9.104031e+09
1997,1.458279e+07,6.755288e+09
1998,1.729041e+07,8.007135e+09


### 인자 정리
- new_X = samples = LHS Sampling으로 생성한 데이터
<!-- - new_metrics = 생성한 데이터에 대해서 TabNET이 예측한 metrics + 원래 metrics (scaling X) -->
- new_metrics_re = 원래 metric (A_metrics) + 생성한 데이터로 예측한 metric (scaling X)
- new_Samples = 원래 config + 생성한 config (scaling X)
- newnewnew = AutoEncoder에 넣을 수 있는 데이터셋

In [19]:
# inversed_new_config = pd.DataFrame(inverse_transformed_samples)
new_X_pd = pd.DataFrame(new_X)
new_X_pd = round(new_X_pd).astype(int)
for i in range(len(new_X_pd.columns)):
    new_X_pd.rename(columns={new_X_pd.columns[i]: A_config.columns[i]}, inplace=True)   
    
new_Samples = pd.concat([A_config,new_X_pd] , axis=0)

In [20]:
new_Samples = new_Samples.reset_index()
new_Samples = new_Samples.drop(['index'], axis=1)
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,38220,6680916,637000,637000,1,6680916,683973542,4,19,...,333977,637000,63845,6370,41,10437,683973914,83865,83865,1
1996,1,56220,9825415,937000,937000,1,9825415,1006096089,7,28,...,491259,937000,93725,9370,60,15352,1006096154,122879,122879,1
1997,1,41700,7288852,695000,695000,1,7288852,746250568,5,21,...,364385,695000,69622,6950,45,11387,746250880,91407,91407,1
1998,1,49440,8640987,824000,824000,1,8640987,884763263,6,25,...,432016,824000,82470,8240,53,13500,884763443,108184,108184,1


In [21]:
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,38220,6680916,637000,637000,1,6680916,683973542,4,19,...,333977,637000,63845,6370,41,10437,683973914,83865,83865,1
1996,1,56220,9825415,937000,937000,1,9825415,1006096089,7,28,...,491259,937000,93725,9370,60,15352,1006096154,122879,122879,1
1997,1,41700,7288852,695000,695000,1,7288852,746250568,5,21,...,364385,695000,69622,6950,45,11387,746250880,91407,91407,1
1998,1,49440,8640987,824000,824000,1,8640987,884763263,6,25,...,432016,824000,82470,8240,53,13500,884763443,108184,108184,1


### NEW_IDEA
### AE Train set에 추가 (1000 + 10000) AUG == new_Samples

In [22]:
for i in range(len(new_X_pd.columns)):
    new_X_pd.rename(columns={new_X_pd.columns[i]: A_config.columns[i]}, inplace=True)   
    
new_Samples = pd.concat([A_config,new_X_pd] , axis=0)

new_Samples = new_Samples.reset_index()
new_Samples = new_Samples.drop(['index'], axis=1)
# new_Samples

newnewwnew = pd.concat([new_Samples, new_metrics_re], axis = 1)

In [23]:
new_metrics_re

,tps,latency
0,2.166658e-01,2.084740e+07
1,3.222207e-01,1.274494e+07
2,3.789313e+03,1.076000e+03
3,1.666666e-01,1.251055e+07
4,2.666662e-01,1.068620e+07
...,...,...
1995,1.336532e+07,6.192648e+09
1996,1.966208e+07,9.104031e+09
1997,1.458279e+07,6.755288e+09
1998,1.729041e+07,8.007135e+09


In [24]:
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,38220,6680916,637000,637000,1,6680916,683973542,4,19,...,333977,637000,63845,6370,41,10437,683973914,83865,83865,1
1996,1,56220,9825415,937000,937000,1,9825415,1006096089,7,28,...,491259,937000,93725,9370,60,15352,1006096154,122879,122879,1
1997,1,41700,7288852,695000,695000,1,7288852,746250568,5,21,...,364385,695000,69622,6950,45,11387,746250880,91407,91407,1
1998,1,49440,8640987,824000,824000,1,8640987,884763263,6,25,...,432016,824000,82470,8240,53,13500,884763443,108184,108184,1


In [25]:
newnewwnew

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,2.166658e-01,2.084740e+07
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,3.222207e-01,1.274494e+07
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3.789313e+03,1.076000e+03
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,1.666666e-01,1.251055e+07
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,2.666662e-01,1.068620e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,38220,6680916,637000,637000,1,6680916,683973542,4,19,...,63845,6370,41,10437,683973914,83865,83865,1,1.336532e+07,6.192648e+09
1996,1,56220,9825415,937000,937000,1,9825415,1006096089,7,28,...,93725,9370,60,15352,1006096154,122879,122879,1,1.966208e+07,9.104031e+09
1997,1,41700,7288852,695000,695000,1,7288852,746250568,5,21,...,69622,6950,45,11387,746250880,91407,91407,1,1.458279e+07,6.755288e+09
1998,1,49440,8640987,824000,824000,1,8640987,884763263,6,25,...,82470,8240,53,13500,884763443,108184,108184,1,1.729041e+07,8.007135e+09


### 원래 sample + 생성한 sample에 대해 AE가 되는지 확인

In [26]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import  TensorDataset, DataLoader

X_train, X_test = train_test_split(newnewwnew, test_size=0.2, shuffle=True)

scaler = MinMaxScaler()
# new_train = pd.concat([X_train,full_samples])
X_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)


dataset_tr = TensorDataset(torch.tensor(X_scaler))
dataset_te = TensorDataset(torch.tensor(X_test_scaler))

trainloader = torch.utils.data.DataLoader(dataset_tr, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset_te, batch_size=128, shuffle=True)


In [27]:
### 원본

import torch.nn as nn

class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = nn.Sequential(
        # nn.Linear(140,64),
        # nn.ReLU(),
        # nn.Dropout(p=0.3),
        # nn.Linear(64,32),
        # nn.ReLU(),
        # nn.Linear(32,16), #잠재변수 15개로 줄임 
        # nn.Sigmoid())
        nn.Linear(140,128),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,32), #잠재변수 15개로 줄임 
        nn.Sigmoid())
    
    self.decoder = nn.Sequential(
        # nn.Linear(16,32),
        # nn.ReLU(),
        # nn.Linear(32,64),
        # nn.ReLU(),
        # nn.Linear(64, 140), 
        # nn.Sigmoid()
        
        nn.Linear(32,64),
        nn.ReLU(),
        # nn.Dropout(p=0.2),
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Linear(128, 140), 
        nn.Sigmoid()
    )
  
  #인코더와 디코더 연산을 차례대로 수행하도록 설정 
  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [28]:
# from torch import device
from torch import optim
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model =Autoencoder().to(device)
model = Autoencoder().to(device)
# model = Autoencoder()


critertion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#lr=1e-4
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainloss = []
validationloss = []
epoch_list = []

for epoch in range(3000):
  running_loss = 0
  model.train()
  
  for data in trainloader:
    inputs = data[0].float().to(device)
    optimizer.zero_grad()
    outputs = model(inputs) 
    loss = critertion(inputs, outputs)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
  train_loss = running_loss / len(trainloader)
  trainloss.append(train_loss)
  # epoch_list.append(epoch)
  
  if epoch % 300 == 0:
    total_val_loss = 0
    model.eval()
    with torch.no_grad():
      running_loss = 0
      
      for data in testloader:
        inputs = data[0].float().to(device)
        # optimizer.zero_grad()
        outputs = model(inputs) 
        loss = critertion(inputs, outputs)
        running_loss += loss.item()
        
      total_val_loss = running_loss / len(testloader)
      validationloss.append(total_val_loss)
        
    print('[%d] tr_loss : %.3f | val_loss : %.3f' %(epoch +1, train_loss, total_val_loss))

[1] tr_loss : 0.127 | val_loss : 0.123
[301] tr_loss : 0.057 | val_loss : 0.070
[601] tr_loss : 0.048 | val_loss : 0.063
[901] tr_loss : 0.043 | val_loss : 0.063
[1201] tr_loss : 0.040 | val_loss : 0.057
[1501] tr_loss : 0.038 | val_loss : 0.059
[1801] tr_loss : 0.036 | val_loss : 0.067
[2101] tr_loss : 0.034 | val_loss : 0.067
[2401] tr_loss : 0.034 | val_loss : 0.070
[2701] tr_loss : 0.033 | val_loss : 0.059


In [35]:
inputs.shape , outputs.shape

(torch.Size([48, 140]), torch.Size([48, 140]))

### Optimization in latent space
- Prediction

In [29]:
import torch
import torch.nn as nn
import numpy as np

for data in testloader:
    inputs = data[0].float().to(device)
    with torch.no_grad():
        encoded_vector = model.encoder(inputs)
        print("Encoded Latent Vector:", encoded_vector)


Encoded Latent Vector: tensor([[0.9940, 0.9480, 0.0065,  ..., 0.0111, 0.9807, 0.8710],
        [0.7845, 0.4681, 0.7816,  ..., 0.5802, 0.5269, 0.4641],
        [0.4795, 0.4677, 0.6251,  ..., 0.4131, 0.4270, 0.0974],
        ...,
        [0.3721, 0.2750, 0.7064,  ..., 0.6670, 0.3742, 0.3439],
        [0.3802, 0.2647, 0.6917,  ..., 0.6657, 0.3483, 0.3280],
        [0.3403, 0.6997, 0.4889,  ..., 0.7890, 0.4430, 0.2186]],
       device='cuda:0')
Encoded Latent Vector: tensor([[0.1266, 0.8359, 0.9595,  ..., 0.0481, 0.9405, 0.0103],
        [0.3544, 0.5226, 0.4824,  ..., 0.5451, 0.7866, 0.0241],
        [0.3750, 0.2689, 0.6973,  ..., 0.6624, 0.3611, 0.3352],
        ...,
        [0.9842, 0.7880, 0.0806,  ..., 0.8411, 0.2503, 0.3484],
        [0.4435, 0.2409, 0.1079,  ..., 0.7016, 0.1443, 0.0633],
        [0.6526, 0.4267, 0.2446,  ..., 0.4830, 0.7610, 0.2683]],
       device='cuda:0')
Encoded Latent Vector: tensor([[0.3995, 0.2173, 0.6958,  ..., 0.6877, 0.2861, 0.3344],
        [0.3827, 0.2235

In [267]:
encoded_vector

tensor([[9.8952e-01, 2.5966e-01, 2.7011e-01,  ..., 1.4159e-01, 1.6972e-03,
         7.9316e-01],
        [3.1741e-03, 1.1137e-01, 5.6049e-01,  ..., 5.4681e-03, 1.8446e-01,
         5.5831e-04],
        [9.7445e-01, 1.6144e-02, 9.7442e-01,  ..., 2.5849e-01, 1.3225e-02,
         9.2028e-01],
        ...,
        [9.0231e-01, 5.7670e-01, 9.7765e-01,  ..., 9.9574e-01, 1.0079e-05,
         2.9574e-01],
        [8.2895e-01, 8.0788e-03, 9.9222e-01,  ..., 1.6199e-01, 2.2575e-03,
         9.7039e-01],
        [3.0056e-01, 9.3433e-01, 3.9254e-01,  ..., 5.5390e-02, 3.5257e-01,
         9.6453e-01]], device='cuda:0')

In [268]:
print(inputs.shape)
print(encoded_vector.shape)

torch.Size([64, 140])
torch.Size([64, 32])


In [38]:
latent_space = encoded_vector
print(latent_space.shape)

torch.Size([44, 32])


### 다시 확인

In [223]:
### test에 사용한 real 데이터의 metric값


for data in testloader:
    inputs = data[0].float().to(device)
    inputs_pd = pd.DataFrame(inputs.cpu().numpy())
    new_metric = inputs_pd.iloc[:,[138,139]]
    # optimizer.zero_grad()
    # print(new_metric)

In [ ]:
new_metric

In [222]:
new_metric.shape

(44, 2)

In [180]:
inputs.shape

torch.Size([48, 140])

In [ ]:
new_metric

### TabNet (Optimization과정에서)

In [66]:
samples.shape

(500, 138)

In [67]:
newnewwnew

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,2.166658e-01,2.084740e+07
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,3.222207e-01,1.274494e+07
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3.789313e+03,1.076000e+03
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,1.666666e-01,1.251055e+07
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,2.666662e-01,1.068620e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,12121,2121392,202000,202000,0,2121392,216895848,1,6,...,20519,2021,14,3310,216896666,27294,27294,0,3.757086e+05,1.094563e+10
1496,1,36480,6376948,608000,608000,1,6376948,652835029,4,18,...,60957,6080,39,9961,652835430,80093,80093,1,1.095164e+11,-3.031297e+13
1497,0,25801,4511212,430000,430000,0,4511212,461708984,3,13,...,43228,4301,28,7045,461709568,56945,56945,0,7.745556e+10,-2.143968e+13
1498,0,14161,2477769,236000,236000,0,2477769,253403070,2,7,...,23906,2361,16,3867,253403853,31715,31715,0,4.180892e+05,1.219115e+10


In [30]:
### 원래 데이터 + LHS Sampling data 차원을 32로 줄인 latent space 구하기
### X = latent config (# 1500) , Y = metrics (# 2)

import torch
import pandas as pd
import numpy as np

scaler_x = MinMaxScaler()
samples_scaler = scaler_x.fit_transform(newnewwnew)

# samples_np = samples_scaler.values
samples_torch = torch.Tensor(samples_scaler).to(device)

with torch.no_grad():
    encoded_vector_BO = model.encoder(samples_torch)
    print("BO'S Encoded Latent Vector:", encoded_vector_BO)

BO'S Encoded Latent Vector: tensor([[0.2533, 0.9017, 0.3771,  ..., 0.7261, 0.8413, 0.2965],
        [0.6064, 0.3052, 0.2440,  ..., 0.0885, 0.3436, 0.8677],
        [0.4014, 0.7224, 0.7219,  ..., 0.5806, 0.7355, 0.2834],
        ...,
        [0.9729, 0.6646, 0.0324,  ..., 0.0251, 0.9164, 0.7158],
        [0.9539, 0.7571, 0.0281,  ..., 0.0422, 0.9089, 0.6395],
        [0.9678, 0.6310, 0.0433,  ..., 0.0239, 0.8954, 0.6385]],
       device='cuda:0')


In [308]:
encoded_vector_BO.shape

torch.Size([1600, 32])

In [309]:
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,11000,1818624,880000,444000,0,1851392,180355072,1,15,...,304384,850000,58800,4000,53,12800,352321536,53248,6144,0
1,0,31000,1097728,455000,80000,1,7929856,880803840,7,1,...,449792,480000,90800,2000,26,16384,192937984,86016,94208,0
2,0,57000,9863168,740000,860000,1,7258112,750780416,1,5,...,170496,790000,23600,4000,14,9472,629145600,118784,73728,1
3,0,44000,5832704,810000,313000,0,737280,964689920,6,4,...,76800,160000,58000,6000,5,2816,763363328,61440,2048,0
4,0,45000,3948544,575000,678000,0,8929280,943718400,4,12,...,149760,80000,69600,4000,57,3584,910163968,57344,14336,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,1,40100,7009341,668333,668333,1,7009341,717617452,5,20,...,350404,668333,66966,6684,43,10950,717617792,87939,87939,1
1596,0,1501,266138,25000,25000,0,266138,26843546,0,1,...,13123,25000,2890,251,3,410,26844544,4275,4275,0
1597,0,24601,4301578,410000,410000,0,4301578,440234148,3,12,...,214968,410000,41236,4101,27,6717,440234752,54344,54344,0
1598,0,21301,3725087,355000,355000,0,3725087,381178348,2,11,...,186133,355000,35758,3551,23,5816,381179008,47191,47191,0


In [31]:
# encoded_vector_BO.shape
new_metrics_re

,tps,latency
0,2.166658e-01,2.084740e+07
1,3.222207e-01,1.274494e+07
2,3.789313e+03,1.076000e+03
3,1.666666e-01,1.251055e+07
4,2.666662e-01,1.068620e+07
...,...,...
1995,1.336532e+07,6.192648e+09
1996,1.966208e+07,9.104031e+09
1997,1.458279e+07,6.755288e+09
1998,1.729041e+07,8.007135e+09


In [32]:
### TabNet
### X = encoded_vector_BO (Scaling O) , Y = metrics (Scaling X)

from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X_latent = np.array(encoded_vector_BO.cpu().numpy())
Y_latent = np.array(new_metrics_re)

lt_X_train, lt_X_test, lt_y_train, lt_y_test = train_test_split(X_latent,Y_latent,test_size=0.3, shuffle=True)
Y_scaler = MinMaxScaler().fit(lt_y_train)

scaled_lt_y_train = Y_scaler.transform(lt_y_train)
scaled_lt_y_test = Y_scaler.transform(lt_y_test)

# Tabnet 모델 생성
lt_regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) 

# 모델 학습
lt_regressor.fit(X_train=lt_X_train, y_train=scaled_lt_y_train,
              eval_set=[(lt_X_test, scaled_lt_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])

# 테스트 데이터로 예측
lt_predictions = lt_regressor.predict(lt_X_test)

# 성능 평가
print('BEST VALID SCORE : ', lt_regressor.best_cost)
print('R2 SCORE : ' , r2_score(scaled_lt_y_test, lt_predictions))

# print('MSE_SCORE : ', test_score)


epoch 0  | loss: 0.23952 | val_0_mse: 0.01395 |  0:00:00s
epoch 10 | loss: 0.00463 | val_0_mse: 0.00694 |  0:00:01s
epoch 20 | loss: 0.00307 | val_0_mse: 0.00644 |  0:00:03s
epoch 30 | loss: 0.00221 | val_0_mse: 0.00647 |  0:00:05s
epoch 40 | loss: 0.00178 | val_0_mse: 0.00324 |  0:00:07s
epoch 50 | loss: 0.00221 | val_0_mse: 0.0018  |  0:00:09s
epoch 60 | loss: 0.00172 | val_0_mse: 0.00176 |  0:00:11s
epoch 70 | loss: 0.00178 | val_0_mse: 0.00131 |  0:00:12s
epoch 80 | loss: 0.00162 | val_0_mse: 0.00134 |  0:00:14s
epoch 90 | loss: 0.00157 | val_0_mse: 0.00123 |  0:00:16s
epoch 100| loss: 0.00164 | val_0_mse: 0.00127 |  0:00:18s
epoch 110| loss: 0.00159 | val_0_mse: 0.00118 |  0:00:20s
epoch 120| loss: 0.00129 | val_0_mse: 0.00153 |  0:00:22s
epoch 130| loss: 0.00127 | val_0_mse: 0.00116 |  0:00:24s
epoch 140| loss: 0.00147 | val_0_mse: 0.0011  |  0:00:26s
epoch 150| loss: 0.00152 | val_0_mse: 0.0014  |  0:00:28s
epoch 160| loss: 0.0016  | val_0_mse: 0.00185 |  0:00:30s
epoch 170| los

In [312]:
lt_X_train , lt_X_test

(array([[9.2988219e-03, 9.8269504e-01, 3.0284757e-03, ..., 1.0204697e-03,
         1.6143235e-03, 7.7340484e-02],
        [6.4069223e-01, 6.0610235e-01, 7.6821381e-01, ..., 4.0478840e-01,
         5.4566270e-01, 9.8787695e-02],
        [4.7050419e-01, 3.2736167e-02, 2.3777217e-01, ..., 9.0984350e-01,
         3.9653379e-01, 9.1868657e-01],
        ...,
        [6.7375129e-01, 1.9076240e-03, 4.7824016e-01, ..., 9.9345464e-01,
         8.9418632e-01, 9.8305005e-01],
        [8.4713668e-01, 1.3203745e-01, 6.5317643e-01, ..., 9.7561532e-01,
         8.8524151e-01, 8.1416243e-01],
        [4.3462822e-01, 1.6756850e-06, 5.2658200e-02, ..., 9.9994111e-01,
         9.9607569e-01, 9.9994588e-01]], dtype=float32),
 array([[8.3768201e-01, 9.5556426e-01, 5.9908164e-01, ..., 6.6146493e-01,
         2.7353358e-02, 5.4459226e-01],
        [3.1627559e-03, 9.9825102e-01, 8.2637806e-04, ..., 2.5457304e-04,
         7.5672120e-05, 1.6813347e-02],
        [7.5694168e-01, 8.2544058e-05, 7.5614625e-01, ...,

In [ ]:
###Gradient Boosting Regressor

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, precision_score
import numpy as np

X_latent = np.array(encoded_vector_BO.cpu().numpy())
Y_latent = np.array(metrics)

lt_X_train, lt_X_test, lt_y_train, lt_y_test = train_test_split(X_latent,Y_latent,test_size=0.2, shuffle=True)
# X_scaler = MinMaxScaler().fit(lt_X_train)
Y_scaler = MinMaxScaler().fit(lt_y_train)

# scaled_lt_X_train = X_scaler.transform(lt_X_train)
# scaled_lt_X_test = X_scaler.transform(lt_X_test)
scaled_lt_y_train = Y_scaler.transform(lt_y_train)
scaled_lt_y_test = Y_scaler.transform(lt_y_test)

# xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
#                            colsample_bytree=1, max_depth=7)

estimator = GradientBoostingRegressor(
                loss = 'squared_error',
                criterion= 'squared_error',
                n_estimators= 5000,
                min_samples_leaf= 20,
                max_depth= 1,
                learning_rate= 0.001,
                random_state=0
                
)

my_model_pred = MultiOutputRegressor(estimator = estimator).fit(lt_X_train, scaled_lt_y_train)


pred = my_model_pred.predict(lt_X_test)

#rmse
# print('정확도 : ', pred.score(scaled_lt_y_test, pred))
print('R2_SCORE : ', r2_score(scaled_lt_y_test, pred))
# print('PCC : ', precision_score(y_test, pred, average='weighted'))

# rmse r2 pcc




In [235]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score

# Gradient Boosting Regressor 모델 설정
estimator = GradientBoostingRegressor()

# MultiOutputRegressor로 감싸서 다중 출력 회귀 모델 생성
mod = MultiOutputRegressor(estimator)

# GridSearchCV를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'estimator__max_depth' : [3,2,1],
    'estimator__min_samples_leaf' : [12,20],
}


# GridSearchCV 객체 생성
grid_search = GridSearchCV(mod, param_grid, cv=5, scoring=make_scorer(r2_score), n_jobs=-1)

# 모델 학습 및 최적 하이퍼파라미터 탐색
grid_search.fit(lt_X_train, scaled_lt_y_train)

# 최적 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)
print("Best R2 Score:", grid_search.best_score_)


Best Hyperparameters: {'estimator__max_depth': 1, 'estimator__min_samples_leaf': 20}
Best R2 Score: -0.05175495580114471


In [185]:
lt_X_test.shape, lt_y_test.shape

((200, 32), (200, 2))

In [187]:
lt_X_train

array([[0.9233435 , 0.54796135, 0.68685925, ..., 0.10056241, 0.48042297, 0.3759139 ],
       [0.3882139 , 0.85031134, 0.21245512, ..., 0.7249691 , 0.43180606, 0.4932873 ],
       [0.9073519 , 0.7776825 , 0.62872344, ..., 0.13256091, 0.7274332 , 0.9297663 ],
       ...,
       [0.8038537 , 0.3006459 , 0.5620582 , ..., 0.15933311, 0.37992233, 0.1274555 ],
       [0.84681034, 0.40041837, 0.6011857 , ..., 0.1869969 , 0.3720844 , 0.81130284],
       [0.1137334 , 0.17089851, 0.5488297 , ..., 0.56721836, 0.48357654, 0.8952433 ]], dtype=float32)

In [215]:
scaled_lt_y_train

array([[7.03583127e-01, 2.20477918e-05],
       [3.60321151e-05, 4.99019742e-01],
       [2.66904401e-05, 5.60341822e-01],
       ...,
       [3.60321827e-05, 8.45218444e-01],
       [1.86832092e-05, 7.72725438e-01],
       [2.00179402e-05, 5.49019679e-01]])

In [161]:
# predictions, scaled_y_test
from sklearn.metrics import r2_score

# r2_score(lt_predictions, lt_y_test)
r2_score(scaled_lt_y_test, lt_predictions)



0.9878279360193556

In [35]:
latent_pd = pd.DataFrame(encoded_vector_BO.cpu().numpy())
latent_pd_T = latent_pd.T
latent_pd_T.index
# latent_pd_T

RangeIndex(start=0, stop=32, step=1)

In [36]:
latent_pd_T

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.253330,0.606402,0.401374,0.914206,0.863556,0.057079,0.565681,0.201260,0.444469,0.402211,...,0.373175,0.967510,0.994034,0.356191,0.829738,0.970652,0.992164,0.972857,0.953884,0.967753
1,0.901733,0.305204,0.722434,0.760502,0.882291,0.990819,0.897042,0.571902,0.567398,0.586622,...,0.111082,0.897330,0.622741,0.189712,0.604114,0.608578,0.937937,0.664607,0.757107,0.631019
2,0.377125,0.244011,0.721946,0.007261,0.254253,0.001726,0.302803,0.271402,0.402253,0.115390,...,0.864722,0.014627,0.042997,0.740845,0.095920,0.041410,0.006972,0.032445,0.028120,0.043294
3,0.428444,0.082144,0.671512,0.167016,0.422887,0.055890,0.828064,0.957496,0.394576,0.160309,...,0.071778,0.704972,0.999185,0.082801,0.944673,0.986830,0.726886,0.982336,0.881729,0.979464
4,0.185774,0.995883,0.276281,0.120691,0.650381,0.993746,0.427634,0.010206,0.774777,0.132526,...,0.131928,0.975092,0.999185,0.213104,0.937361,0.993208,0.988391,0.991055,0.981661,0.986826
5,0.914632,0.041765,0.663017,0.714148,0.594114,0.158215,0.089978,0.889468,0.197893,0.252099,...,0.920129,0.035947,0.030996,0.683773,0.093917,0.058249,0.022341,0.044353,0.040419,0.076535
6,0.758504,0.943018,0.284934,0.019251,0.195638,0.209670,0.059579,0.154297,0.269893,0.682491,...,0.083724,0.530051,0.253939,0.114342,0.312418,0.295190,0.681475,0.295936,0.389281,0.278649
7,0.596180,0.056833,0.775608,0.980428,0.952649,0.221732,0.964979,0.865333,0.726587,0.189156,...,0.064377,0.912158,0.829347,0.149795,0.848108,0.896168,0.925129,0.870180,0.888685,0.884069
8,0.441174,0.768973,0.433746,0.470466,0.417272,0.098673,0.943374,0.925307,0.315068,0.807808,...,0.900512,0.054391,0.002653,0.746720,0.080038,0.010708,0.031750,0.010486,0.025797,0.013068
9,0.240139,0.260385,0.532047,0.016383,0.403148,0.646370,0.537406,0.731081,0.370897,0.342090,...,0.859997,0.016691,0.004292,0.660559,0.019247,0.006409,0.008898,0.005559,0.013991,0.007927


In [39]:
### BO 코드
from sklearn.preprocessing import MinMaxScaler
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

class BO(object):
    def __init__(self, iteration, configs, metrics, config_info_path=None):
        self.iteration = iteration
        self.configs = configs
        self.metrics = metrics
        self.config_info_path = config_info_path
        # self.min_max_same_knobs = []
        
        self._get_config_info()
        self._init_pbounds()
    
    def _get_config_info(self):
        if self.config_info_path is None:
            self.config_info = pd.read_csv('/home/sein/mk_config/Knob_Information_MySQL_v5.7.csv', index_col=0)
        else:
            self.config_info = pd.read_csv(self.config_info_path, index_col=0)
        
        # if self.top_z_knob is not None:
        #     self.config_info = self.config_info.loc[self.top_z_knob]
        #     self.configs = self.configs[self.top_z_knob]
            

#     def _get_history(self):
#         self.history_configs = self.smac.runhistory.get_configs()
    
    def _init_pbounds(self):
        self.pbounds = {}
        
        for v in latent_pd_T.index:
            # self.pbounds[v] = (0,18)
            self.pbounds[str(v)] = (0, 1)
            
    
    def train_regression_model(self):
        X_all = np.array(self.configs)
        Y_all = np.array(self.metrics)
        
        cnt = 0
        while(True):
            X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)         
            
            self.X_scaler = MinMaxScaler().fit(X_train)
            self.y_scaler = MinMaxScaler().fit(y_train)

            # scaled_X_train = self.X_scaler.transform(X_train)
            # scaled_X_test = self.X_scaler.transform(X_test)
            scaled_y_train = self.y_scaler.transform(y_train)
            scaled_y_test = self.y_scaler.transform(y_test)

            estimator = TabNetRegressor(verbose = 100, seed = 42, optimizer_fn=torch.optim.AdamW)

            estimator.fit(X_train=X_train, y_train=scaled_y_train,
              eval_set=[(X_test, scaled_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])
            pred = estimator.predict(X_test)
            accuracy = mean_squared_error(y_pred = pred, y_true = scaled_y_test)
            # r2score = r2_score(scaled_y_test, pred)

            cnt += 1
            print(cnt, accuracy)
            if accuracy < 0.05 or cnt > 10:
                break


            # print('BEST VALID SCORE : ', regressor.best_cost)

        self.model = estimator
    
    def _target_function(self, **kwargs):
        x = np.fromiter(kwargs.values(), dtype=float)        
        scaled_X = self.X_scaler.transform([x])
        res = self.model.predict(scaled_X)
        res = res[:,0] / res[:,1]
        # for name in self.config_info.index:
        #     knob = self.config_info.loc[name]
        #     knob_min = knob['raw_min']
        #     knob_max = knob['raw_max']
            
            # if self.reduce_search_space:
            #     knob_min = self.best_configs[name].min()
            #     knob_max = self.best_configs[name].max()
            #     if knob_min == knob_max:
            #         self.min_max_same_knobs.append(name)
            #         knob_max += 1
            # with torch.no_grad():
            #     scaled_X_t = torch.tensor(scaled_X)
            #     scaled_X_t = scaled_X_t.to('cuda:0')
            #     if model.decoder(scaled_X_t) < knob_min:
            #         res == 0
            #     elif model.decoder(scaled_X_t) > knob_max:
            #         res == 0
        
        
        return res.squeeze()
        # return res
    
    
    def tune(self):
        self.optimizer = BayesianOptimization(f=self._target_function, pbounds=self.pbounds, verbose=2, random_state=2)
        # self.optimizer = BayesianOptimization(f=self._target_function, verbose=2, random_state=2)
        self.acquisition_function = UtilityFunction(kind="ei", xi=0.01)
        self.optimizer.maximize(n_iter=self.iteration, acquisition_function=self.acquisition_function)
    
    
    def plot_history(self):
        self.y_obs = - np.array([res["target"] for res in self.optimizer.res])
        
        self.his_inc = []
        inc = np.inf
        ## Get minimum results on each iteration
        for res in self.y_obs:
            if res < inc:
                inc = res
            self.his_inc.append(inc)
#             res.append(his_res)

        plt.plot(self.his_inc)
        plt.xlabel('Iteration')
        plt.ylabel('result')
        plt.show()

### 밑에 다 돌린거고 decoding해서 값 확인해보기

In [40]:
################metric 확인, metric 확인, metric 확인, metric 확인
tuner1 = BO(iteration=1500, 
           configs=encoded_vector_BO.cpu().numpy(),
           metrics=new_metrics_re
           )
tuner1.train_regression_model()
tuner1.tune()
# tuner1.plot_history()

epoch 0  | loss: 0.25049 | val_0_mse: 0.01381 |  0:00:00s
epoch 100| loss: 0.00044 | val_0_mse: 7e-05   |  0:00:18s

Early stopping occurred at epoch 186 with best_epoch = 136 and best_val_0_mse = 4e-05
1 3.757032169231846e-05
|   iter    |  target   |     0     |     1     |    10     |    11     |    12     |    13     |    14     |    15     |    16     |    17     |    18     |    19     |     2     |    20     |    21     |    22     |    23     |    24     |    25     |    26     |    27     |    28     |    29     |     3     |    30     |    31     |     4     |     5     |     6     |     7     |     8     |     9     |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

ValueError: Input y contains NaN.

In [174]:
ex_data = [0.2515,  0.3851, 0.1983, 0.2072, 0.04203, 0.6034, 0.7527, 0.309, 0.01957, 0.107,
           0.6571, 0.7406, 0.9737, 0.1054, 0.5471, 0.467, 0.3135, 0.9268, 0.1925, 0.1914, 0.7365,
           0.4656, 0.9693, 0.5216, 0.2464, 0.4485, 0.282, 0.4303, 0.4483, 0.5755, 0.1848, 0.99]

In [221]:
ex_data = [0.2168, 0.0, 0.138, 1.0, 0.3111, 0.333, 0.6079, 0.1281, 0.0, 0.2796, 0.3718, 0.3238, 0.5304,
           0.474, 0.1646, 0.5807, 0.9193, 0.09729, 0.1823, 0.1282, 0.3789, 0.4876, 0.9011, 0.1251, 0.3144,
           0.04871, 0.8719, 1.0, 0.5318, 0.5375, 0.7821, 0.865]

In [41]:
ex_data = [0.8043, 0.4696, 0.982, 0.6699, 0.7243, 0.5484, 0.2724, 0.7798, 0.03929,
           0.8069, 0.8017, 0.5946, 0.2589, 0.09975, 0.06845, 0.1965, 0.08088, 0.07003, 0.08094,
           0.4855, 0.8332, 0.5626, 0.6881, 0.9309, 0.9061, 0.6282, 0.06681, 0.5079,
           0.5546, 0.1638, 0.7791, 0.03581]

In [41]:
ex_data = [0.6213, 0.8054, 0.8326, 0.936, 0.7955, 0.6943, 0.3554, 0.2464, 0.8895,
           0.00615, 0.8281, 0.2909, 0.4572, 0.7897, 0.5804, 0.4555, 0.3784, 0.5546,
           0.7186, 0.4245, 0.2149, 0.955, 0.4766, 0.3993, 0.6205, 0.4485, 0.8087,
           0.555, 0.2683, 0.313, 0.5913, 0.8213]

In [42]:
ex_data = torch.tensor(ex_data)
ex_data = ex_data.to('cuda:0')  # ex_data를 GPU로 이동

with torch.no_grad():
    decode_value = model.decoder(ex_data)
    print("Decoded Value:", decode_value)

Decoded Value: tensor([2.7839e-04, 4.0109e-01, 4.0013e-01, 2.6214e-01, 5.9080e-01, 5.0324e-01,
        6.5841e-01, 5.7817e-01, 2.8931e-01, 5.4068e-01, 9.9324e-01, 7.0188e-01,
        3.1349e-01, 1.9275e-02, 6.8353e-01, 4.0811e-01, 7.2324e-01, 9.7091e-01,
        4.5705e-01, 2.9855e-01, 2.7486e-01, 4.7988e-01, 3.9624e-01, 4.6286e-01,
        6.0591e-01, 6.0115e-01, 3.8855e-01, 4.6597e-01, 6.5717e-01, 6.1959e-01,
        9.9111e-01, 1.3310e-02, 1.1084e-01, 6.6937e-01, 7.4448e-04, 5.5355e-01,
        9.9264e-03, 3.5375e-02, 5.0302e-01, 4.7123e-01, 5.7412e-01, 3.4057e-01,
        4.8784e-01, 4.0894e-01, 3.6165e-01, 4.8625e-01, 4.7295e-01, 4.8753e-01,
        5.8816e-01, 4.4720e-01, 4.9585e-01, 4.9248e-01, 5.7902e-01, 4.5915e-01,
        4.5809e-01, 3.2542e-01, 7.1879e-01, 4.8760e-01, 9.8301e-01, 3.1795e-01,
        9.9335e-01, 5.8510e-01, 5.0278e-01, 3.4473e-01, 7.5254e-01, 5.2156e-01,
        5.0884e-01, 3.1658e-01, 7.3286e-01, 4.3400e-01, 2.2774e-01, 5.0372e-01,
        9.8609e-01, 9.648

In [43]:
for x in decode_value:
    print(x)

tensor(0.0003, device='cuda:0')
tensor(0.4011, device='cuda:0')
tensor(0.4001, device='cuda:0')
tensor(0.2621, device='cuda:0')
tensor(0.5908, device='cuda:0')
tensor(0.5032, device='cuda:0')
tensor(0.6584, device='cuda:0')
tensor(0.5782, device='cuda:0')
tensor(0.2893, device='cuda:0')
tensor(0.5407, device='cuda:0')
tensor(0.9932, device='cuda:0')
tensor(0.7019, device='cuda:0')
tensor(0.3135, device='cuda:0')
tensor(0.0193, device='cuda:0')
tensor(0.6835, device='cuda:0')
tensor(0.4081, device='cuda:0')
tensor(0.7232, device='cuda:0')
tensor(0.9709, device='cuda:0')
tensor(0.4571, device='cuda:0')
tensor(0.2986, device='cuda:0')
tensor(0.2749, device='cuda:0')
tensor(0.4799, device='cuda:0')
tensor(0.3962, device='cuda:0')
tensor(0.4629, device='cuda:0')
tensor(0.6059, device='cuda:0')
tensor(0.6011, device='cuda:0')
tensor(0.3886, device='cuda:0')
tensor(0.4660, device='cuda:0')
tensor(0.6572, device='cuda:0')
tensor(0.6196, device='cuda:0')
tensor(0.9911, device='cuda:0')
tensor(0

In [44]:
import math


real_values = [math.exp(x) for x in decode_value]  #지수함수 없는 상태

print(real_values)

[1.000278429378128, 1.4934547828632054, 1.4920259898105392, 1.2997142291483355, 1.8054385097881527, 1.6540798574779145, 1.93171502936551, 1.7827675525858937, 1.3355075130451648, 1.7171719792671898, 2.699960967997125, 2.017534142666168, 1.3681969230519906, 1.019461943448249, 1.9808561268759295, 1.5039660789832596, 2.061102494556563, 2.6403569740028767, 1.5794081753046392, 1.3479081257924383, 1.3163449933763955, 1.6158861757101746, 1.4862300379346227, 1.5886182409749274, 1.832924547475418, 1.8242071354389946, 1.474843820039067, 1.5935631489199669, 1.9293305879566736, 1.8581703280775543, 2.694211911137446, 1.0133992869427768, 1.1172128752408017, 1.9530121178605035, 1.0007447590574632, 1.739408715135939, 1.0099758481480057, 1.0360084815179442, 1.6537115630989125, 1.6019572385637177, 1.7755692184820298, 1.405751952570619, 1.6287905276827668, 1.5052151950156079, 1.435698956062626, 1.626210880176993, 1.6047163330952805, 1.6282946522194903, 1.8006801393695033, 1.5639325536994335, 1.64189921985

In [45]:
### 원본

import numpy as np

# real_values 리스트를 NumPy 배열로 변환
real_values = np.array(real_values)
rescaled_actual = scaler.inverse_transform(real_values.reshape(1,-1))

In [46]:
# rescaled_actual.shape
# real_values.shape
rescaled_actual


array([[1.00027843e+00, 8.96067935e+04, 1.56185658e+07, 1.29971423e+06,
        1.80543851e+06, 1.65407986e+00, 2.02516839e+07, 1.90675461e+09,
        9.34855259e+00, 5.15151594e+01, 2.69996097e+00, 2.01753414e+04,
        9.57737846e+00, 1.01946194e+00, 3.56554103e+02, 2.35594912e+01,
        2.06110249e+03, 2.64035697e+00, 6.42113542e+03, 6.89781052e+02,
        1.31634499e+06, 1.60395909e+03, 2.50111521e+10, 7.94309120e+01,
        1.83292455e+00, 1.81143769e+03, 1.47484382e+02, 1.43420683e+01,
        1.44699794e+02, 1.85816175e+05, 2.69421191e+00, 1.01339929e+00,
        1.11721288e+00, 1.85771091e+02, 1.00074476e+00, 1.37433516e+08,
        1.00997585e+00, 1.03600848e+00, 4.06890775e+02, 2.56313158e+01,
        1.69801230e+04, 1.50900891e+09, 5.14925064e+01, 1.94070988e+09,
        1.43134197e+04, 1.61421746e+05, 1.71036737e+09, 1.73354204e+09,
        2.90923952e+04, 1.54750209e+04, 1.62548023e+02, 1.61999803e+02,
        1.78429199e+05, 1.58272387e+07, 2.71562807e+10, 8.307675

In [48]:
pd.options.display.float_format = '{:.9f}'.format #지수함수 없이 출력하는 option

rescaled_actual_pd = pd.DataFrame(rescaled_actual)
rescaled_actual_pd

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,1.000278429,89606.793517009,15618565.750644440,1299714.229148336,1805438.509788153,1.654079857,20251683.881559409,1906754614.604712248,9.348552591,51.515159378,...,142718.614812451,15798.215228254,121.208650732,25273.419623672,1479378145.796419859,207063.406624062,258440.849524386,1.000190731,1064647286.880943775,14366508320340.742187500


In [49]:
# len(A_config.columns)
rescaled_actual_pd.iloc[0][1]

89606.79351700946

In [50]:
for i in range (len(A_config.columns)):
    print('{} = {}'.format(A_config.columns[i], round(rescaled_actual_pd.iloc[0][i])))

automatic_sp_privileges  = 1
back_log  = 89607
binlog_cache_size  = 15618566
binlog_group_commit_sync_delay  = 1299714
binlog_group_commit_sync_no_delay_count  = 1805439
binlog_rows_query_log_events  = 2
binlog_stmt_cache_size  = 20251684
bulk_insert_buffer_size  = 1906754615
default_week_format  = 9
div_precision_increment  = 52
end_markers_in_json  = 3
eq_range_index_dive_limit  = 20175
expire_logs_days  = 10
explicit_defaults_for_timestamp  = 1
flush_time  = 357
ft_min_word_len  = 24
ft_query_expansion_limit  = 2061
general_log  = 3
group_concat_max_len  = 6421
innodb_adaptive_hash_index_parts  = 690
innodb_adaptive_max_sleep_delay  = 1316345
innodb_autoextend_increment  = 1604
innodb_buffer_pool_size  = 25011152101
innodb_change_buffer_max_size  = 79
innodb_cmp_per_index_enabled  = 2
innodb_commit_concurrency  = 1811
innodb_compression_failure_threshold_pct  = 147
innodb_compression_level  = 14
innodb_compression_pad_pct_max  = 145
innodb_concurrency_tickets  = 185816
innodb_deadlo

### Data Prepare (Metric)

In [158]:
inputs

tensor([[0.0000e+00, 1.9999e-01, 2.7472e-01,  ..., 0.0000e+00, 2.5393e-05,
         5.5890e-01],
        [1.0000e+00, 8.3333e-01, 2.4619e-02,  ..., 0.0000e+00, 2.6730e-05,
         5.1992e-01],
        [1.0000e+00, 6.6666e-01, 1.2466e-01,  ..., 1.0000e+00, 7.2171e-05,
         4.9469e-01],
        ...,
        [1.0000e+00, 9.3333e-01, 7.5615e-01,  ..., 1.0000e+00, 1.7374e-05,
         4.3178e-01],
        [1.0000e+00, 4.9984e-02, 8.4838e-01,  ..., 0.0000e+00, 1.8711e-05,
         5.3462e-01],
        [0.0000e+00, 6.4999e-01, 9.5623e-01,  ..., 1.0000e+00, 2.5393e-05,
         5.3893e-01]], device='cuda:0')

In [124]:
latent_pd = pd.DataFrame(latent_space.cpu().numpy())

In [127]:
latent_pd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,1.739686,3.828686,1.652943,4.827423,2.400748,3.672786,0.0,0.0,1.051731,2.765533,2.328941,6.540833,3.163663,4.077005,2.623121,0.0,2.023202,1.931337,2.904802,2.631545,3.034909,2.069007,2.389846,1.221170,0.0,1.757820,2.423846,0.0,3.376463,0.0,0.0,4.204070
1,5.263201,4.982790,3.039382,0.937511,3.944671,3.973503,0.0,0.0,2.720342,3.109445,7.129198,2.421449,2.519826,2.566425,5.555007,0.0,6.304241,4.550933,2.547660,1.511670,4.295227,3.176923,2.138120,2.607466,0.0,6.696853,4.443738,0.0,4.051492,0.0,0.0,4.196002
2,2.766529,3.934936,3.985363,8.824937,2.800091,3.820795,0.0,0.0,3.465561,2.104327,2.924572,1.036899,4.624174,1.789211,3.822195,0.0,3.855425,6.226820,2.086607,2.721859,3.656491,7.108167,1.398559,4.138853,0.0,5.400802,4.325433,0.0,2.952841,0.0,0.0,2.552230
3,2.518008,2.992872,1.849876,3.105592,1.760748,5.047376,0.0,0.0,4.425196,5.215411,5.812379,2.419034,3.874475,3.489141,3.192498,0.0,2.619721,3.345405,3.742889,5.348268,4.671769,2.296950,1.058514,3.009339,0.0,4.251121,5.161115,0.0,3.539448,0.0,0.0,5.888244
4,5.710105,3.119369,5.798205,1.655647,12.115639,3.281065,0.0,0.0,7.373196,2.274594,4.255859,5.133362,3.946908,4.535933,6.791177,0.0,2.498708,5.733562,4.409940,3.117545,5.725585,1.837201,3.865963,12.683617,0.0,3.366748,4.133466,0.0,3.142618,0.0,0.0,3.934982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,4.376726,2.973006,1.929322,1.804700,3.990704,2.082718,0.0,0.0,6.948726,3.968873,5.486206,1.961821,3.856718,4.884863,3.910616,0.0,2.983663,1.855559,5.642768,1.984998,1.719367,0.608689,2.078119,2.691597,0.0,4.484377,2.292664,0.0,1.137102,0.0,0.0,1.345621
396,9.931121,2.924120,7.188254,3.110448,9.753383,5.154171,0.0,0.0,7.669095,2.752270,2.582700,7.504115,3.361691,5.864507,11.000066,0.0,1.853562,7.554099,4.985822,3.244969,3.621013,7.307350,3.714561,14.271664,0.0,7.430314,8.832792,0.0,0.822277,0.0,0.0,4.305506
397,5.663233,7.589723,6.272685,1.718795,4.167714,5.629080,0.0,0.0,9.432640,5.620521,6.807207,3.970119,2.691149,5.694302,13.940556,0.0,2.146896,6.978596,4.964777,2.491496,4.077055,0.278757,5.440994,3.077384,0.0,11.976365,5.347006,0.0,2.691532,0.0,0.0,6.947793
398,4.824186,3.774930,6.257770,4.998737,7.097552,4.055510,0.0,0.0,3.116205,4.473438,5.437254,10.025055,5.286506,2.654964,3.325086,0.0,6.870083,3.196509,4.759892,6.332176,4.249042,2.689435,3.173205,9.702465,0.0,3.561816,0.646306,0.0,6.430585,0.0,0.0,5.452132


In [140]:
latent_pd.max()

0     13.221277
1     11.478708
2     11.184992
3     12.917171
4     12.115639
5      9.768948
6      0.000000
7      0.000000
8     11.189557
9      9.044129
10    11.367462
11    10.141627
12     7.989879
13    11.597285
14    17.549595
15     0.000000
16    12.435203
17    15.429522
18    11.269625
19    10.515278
20    11.656007
21    13.599747
22    13.877734
23    15.734437
24     0.000000
25    15.759394
26    12.802032
27     0.000000
28    10.702847
29     0.000000
30     0.000000
31    10.890145
dtype: float32

### Tabnet type 나눈거

In [14]:
### TabNet
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

X_all = np.array(latent_space.cpu().numpy())
Y_all = np.array(new_metric)

### 데이터 나누기
### boolean에 해당하는 열 정리

discrete_columns = [samples.columns[0], samples.columns[5],samples.columns[10],samples.columns[13],
                    samples.columns[17],samples.columns[24],samples.columns[30],samples.columns[31],
                    samples.columns[32],samples.columns[34],samples.columns[36],samples.columns[37],
                    samples.columns[58],samples.columns[60],samples.columns[64],samples.columns[68],
                    samples.columns[72],samples.columns[73],samples.columns[74],samples.columns[75],
                    samples.columns[77],samples.columns[80],samples.columns[82],samples.columns[83],
                    samples.columns[90],samples.columns[91],samples.columns[92],samples.columns[93],
                    samples.columns[118],samples.columns[123],samples.columns[124],samples.columns[125],
                    samples.columns[126]]

discrete_idxs = [0, 5, 10, 13, 17, 24, 30, 31, 32, 34, 36, 37, 58, 60, 64, 68, 72, 73, 74, 75, 77, 80, 82,
                 83, 90, 91, 92, 93, 118, 123, 124, 125, 126]

discrete_dims = 

X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.2, shuffle=True)
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler = MinMaxScaler().fit(y_train)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)
scaled_y_train = Y_scaler.transform(y_train)
scaled_y_test = Y_scaler.transform(y_test)

# Tabnet 모델 생성
regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) ### Basic

# 모델 학습
regressor.fit(X_train=scaled_X_train, y_train=scaled_y_train,
              eval_set=[(scaled_X_test, scaled_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])

# regressor.fit(X_train=scaled_X_train, y_train=scaled_y_train)



# predictions_array =[]
# CV_score_array    =[]

# CV_score_array.append(regressor.best_cost)
# predictions_array.append(np.expm1(regressor.predict(scaled_X_test)))

# predictions = np.mean(predictions_array,axis=0)

# 테스트 데이터로 예측
# y_pred = rf_regressor.predict(scaled_X_test)
predictions = regressor.predict(scaled_X_test)

test_score = mean_squared_error(y_pred = predictions, y_true = scaled_y_test)
# 성능 평가
print('BEST VALID SCORE : ', regressor.best_cost)
print('MSE_SCORE : ', test_score)


SyntaxError: invalid syntax (1197736205.py, line 27)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

X_all = np.array(new_knobs)
Y_all = np.array(new_metric)

est = XGBRegressor()

param_grid = {
    'n_estimators' : [15000, 20000, 25000],
    'max_depth' : [12,13,14,15],
    'learning_rate' : [0.1, 0.001],  
}

grid_search = GridSearchCV(estimator=est, param_grid=param_grid, scoring='accuracy', cv = 3)

grid_search.fit(X_all,Y_all)

print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

### ㅎㅎ..

In [135]:
#####Shap_value

###XGBRegressor

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
# from sklearn.metrics import explained_variance_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, precision_score
import numpy as np

X_all = np.array(new_knobs)
Y_all = np.array(new_metric)

X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.4, shuffle=True)
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler = MinMaxScaler().fit(y_train)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)
scaled_y_train = Y_scaler.transform(y_train)
scaled_y_test = Y_scaler.transform(y_test)

# xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
#                            colsample_bytree=1, max_depth=7)

estimator = XGBRegressor(
    objective = 'reg:squarederror',
    random_state = 2, n_estimators = 15000, max_depth = 12, learning_rate = 0.01)

my_model_pred = MultiOutputRegressor(estimator = estimator).fit(scaled_X_train, scaled_y_train)


pred = my_model_pred.predict(scaled_X_test)

#rmse
print('RMSE : ',mean_squared_error(scaled_y_test, pred, squared=False))
print('R2_SCORE : ', r2_score(scaled_y_test, pred))
# print('PCC : ', precision_score(y_test, pred, average='weighted'))

# rmse r2 pcc




RMSE :  0.07473983514026226
R2_SCORE :  0.9198065356603133


- Optimization

In [151]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

class BO(object):
    def __init__(self, iteration, configs, metrics,config_info_path=None):
        self.iteration = iteration
        self.configs = configs
        self.metrics = metrics
        self.config_info_path = config_info_path
        # self.reduce_search_space = reduce_search_space
        # self.top_z_knob = top_z_knob
        # self.best_configs = best_configs
        self.min_max_same_knobs = []
        
        self._get_config_info()
        self._init_pbounds()
    
    def _get_config_info(self):
        if self.config_info_path is None:
            self.config_info = pd.read_csv('/home/sein/mk_config/Knob_Information_MySQL_v5.7.csv', index_col=0)
        else:
            self.config_info = pd.read_csv(self.config_info_path, index_col=0)
        
        # if self.top_z_knob is not None:
        #     self.config_info = self.config_info.loc[self.top_z_knob]
        #     self.configs = self.configs[self.top_z_knob]
            

#     def _get_history(self):
#         self.history_configs = self.smac.runhistory.get_configs()
    
    def _init_pbounds(self):
        self.pbounds = {}
        
        for name in self.config_info.index:
            knob = self.config_info.loc[name]
            knob_type = knob['type']
            knob_min = knob['raw_min']
            knob_max = knob['raw_max']
            
            # if self.reduce_search_space:
            #     knob_min = self.best_configs[name].min()
            #     knob_max = self.best_configs[name].max()
            #     if knob_min == knob_max:
            #         self.min_max_same_knobs.append(name)
            #         knob_max += 1
            
            if knob_type == 'boolean' or knob_type == 'integer':
                self.pbounds[name] = (int(knob_min), int(knob_max))
            elif knob_type == 'float':
                self.pbounds[name] = (float(knob_min), float(knob_max))
    
    def train_regression_model(self):
        X_all = np.array(self.configs)
        Y_all = np.array(self.metrics)

        cnt = 0
        while(True):
            X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)
            self.X_scaler = MinMaxScaler().fit(X_train)
            self.y_scaler = MinMaxScaler().fit(y_train)

            scaled_X_train = self.X_scaler.transform(X_train)
            scaled_X_test = self.X_scaler.transform(X_test)
            scaled_y_train = self.y_scaler.transform(y_train)
            scaled_y_test = self.y_scaler.transform(y_test)

            estimator = XGBRegressor(
                objective = 'reg:squarederror',
                random_state = 2, n_estimators = 15000, max_depth = 12, learning_rate = 0.01)

            my_model_pred = MultiOutputRegressor(estimator = estimator).fit(scaled_X_train, scaled_y_train)
            pred = my_model_pred.predict(scaled_X_test)
            accuracy = r2_score(scaled_y_test, pred)

            cnt += 1
            print(cnt, accuracy)
            if accuracy > 0.45 or cnt > 10:
                break

        #rmse
        print('RMSE : ',mean_squared_error(scaled_y_test, pred, squared=False))
        print('R2_SCORE : ', r2_score(scaled_y_test, pred))

        self.model = my_model_pred
    
    def _target_function(self, **kwargs):
        x = np.fromiter(kwargs.values(), dtype=float)        
        scaled_X = self.X_scaler.transform([x])
        
        res = self.model.predict(scaled_X)
        res = res[:,0] / res[:,1]
        
        return res.squeeze()
#         print(y)
        
#         X = pd.DataFrame(config.get_dictionary().values(), index=self.config_info.index).T
#         X = np.array(X)
#         scaled_X = self.X_scaler.transform(X)
        
#         res = self.model.predict(scaled_X)
#         res = res[:,1] / res[:,0]
        
#         return res # a smaller result means better
    
    
    def tune(self):
        self.optimizer = BayesianOptimization(f=self._target_function, pbounds=self.pbounds, verbose=2, random_state=2)
        # self.optimizer = BayesianOptimization(f=self._target_function, verbose=2, random_state=2)
        self.acquisition_function = UtilityFunction(kind="ei", xi=0.01)
        self.optimizer.maximize(n_iter=self.iteration, acquisition_function=self.acquisition_function)
    
    
    def plot_history(self):
        self.y_obs = - np.array([res["target"] for res in self.optimizer.res])
        
        self.his_inc = []
        inc = np.inf
        ## Get minimum results on each iteration
        for res in self.y_obs:
            if res < inc:
                inc = res
            self.his_inc.append(inc)
#             res.append(his_res)

        plt.plot(self.his_inc)
        plt.xlabel('Iteration')
        plt.ylabel('result')
        plt.show()

In [152]:
tuner1 = BO(iteration=150, 
           configs=new_knobs,
           metrics=new_metric
           )
tuner1.train_regression_model()
tuner1.tune()
# tuner1.plot_history()

1 0.9437982956585778
RMSE :  0.05482656103790502
R2_SCORE :  0.9437982956585778
|   iter    |  target   | automa... | back_log  | binlog... | binlog... | binlog... | binlog... | binlog... | bulk_i... | defaul... | div_pr... | end_ma... | eq_ran... | expire... | explic... | flush_... | ft_min... | ft_que... | genera... | group_... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb... | innodb